In [0]:
%sql
-- 1. Cria o Catálogo principal para organizar o projeto
CREATE CATALOG IF NOT EXISTS medalhao;

In [0]:
%sql
-- 2. Define esse catálogo como o padrão para esta sessão
USE CATALOG medalhao;

-- 3. Cria os schemas (databases) bronze e silver dentro dele
CREATE SCHEMA IF NOT EXISTS bronze;
CREATE SCHEMA IF NOT EXISTS silver;

In [0]:
#Criação da sessão Spark
from pyspark.sql import SparkSession

#Funções do Spark para manipulação de dados
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

#Bibliotecas para a chamada da API do Banco Central
import requests
import pandas as pd
from datetime import datetime
import time

In [0]:
catalogo = "medalhao"
schema_bronze = "bronze"

#Define o caminho completo para o arquivo CSV
volume_path = "/Volumes/workspace/default/bronze_olist"

#Arquivo fonte e tabela destino para o primeiro item da lista
source_file_name = "olist_customers_dataset.csv"
target_table_name = "ft_consumidores"

In [0]:
#Lista de todos os 9 trabalhos de ingestão para a camada bronze
#Cada item é um dicionário com o nome do arquivo fonte e o nome da tabela destino!

tabelas_bronze_jobs = [
    {
        "arquivo": "olist_customers_dataset.csv",
        "tabela": "ft_consumidores"
    },
    {
        "arquivo": "olist_geolocation_dataset.csv",
        "tabela": "ft_geolocalizacao"
    },
    {
        "arquivo": "olist_order_items_dataset.csv",
        "tabela": "ft_itens_pedidos"
    },
    {
        "arquivo": "olist_order_payments_dataset.csv",
        "tabela": "ft_pagamentos_pedidos"
    },
    {
        "arquivo": "olist_order_reviews_dataset.csv",
        "tabela": "ft_avaliacoes_pedidos"
    },
    {
        "arquivo": "olist_orders_dataset.csv",
        "tabela": "ft_pedidos"
    },
    {
        "arquivo": "olist_products_dataset.csv",
        "tabela": "ft_produtos"
    },
    {
        "arquivo": "olist_sellers_dataset.csv",
        "tabela": "ft_vendedores"
    },
    {
        "arquivo": "product_category_name_translation.csv",
        "tabela": "dm_categoria_produtos_traducao"
    }
]

print(f"Lista de {len(tabelas_bronze_jobs)} trabalhos definida.")

In [0]:
def ingest_csv_to_bronze(arquivo_fonte, tabela_destino):
    
    full_source_path = f"{volume_path}/{arquivo_fonte}"
    full_target_table_name = f"{catalogo}.{schema_bronze}.{tabela_destino}"
    
    print(f"Processando: {arquivo_fonte} -> {full_target_table_name} ... ", end="")
    
    try:
        #EXTRAIR
        df = (
            spark.read
            .format("csv")
            .option("header", "true")
            .option("inferSchema", "true")
            .load(full_source_path)
        )
        
        #TRANSFORMAR
        df_with_timestamp = df.withColumn("ingestion_timestamp", F.current_timestamp())
        
        #CARREGAR
        (
            df_with_timestamp.write
            .mode("overwrite")  
            .format("delta")
            .saveAsTable(full_target_table_name)
        )
        
        print(f"SUCESSO.") 
        
    except Exception as e:
        print(f"FALHA. Erro: {str(e)}")

In [0]:
print("--- Iniciando o loop de ingestão para a camada Bronze ---")

for job in tabelas_bronze_jobs:
    
    #Chama a função de ingestão para cada item da lista
    ingest_csv_to_bronze(
        arquivo_fonte=job["arquivo"], 
        tabela_destino=job["tabela"]
    )

print("\nLoop de ingestão Bronze concluído")

In [0]:
#Célula de Verificação
#Faz uma consulta simples em cada tabela criada para verificar os dados.

print(f"Iniciando verificação das tabelas no {catalogo}.{schema_bronze}")

for job in tabelas_bronze_jobs:
    tabela_nome_completo = f"{catalogo}.{schema_bronze}.{job['tabela']}"
    
    print(f"\nVerificando tabela: {tabela_nome_completo}")
    
    try:
        #Lê a tabela Delta mostra 5 linhas
        df_verificacao = spark.read.table(tabela_nome_completo)
        display(df_verificacao.limit(5))
        
    except Exception as e:
        #Se a tabela não existir
        print(f"FALHA ao verificar. Tabela '{tabela_nome_completo}' não encontrada. Erro: {str(e)}")

print("\nVerificação concluída")

In [0]:
#Definindo a função de ingestão da API

def ingest_cotacao_dolar_to_bronze(data_inicio, data_fim, catalogo_destino, schema_destino):
    
    api_url = f"https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@dataInicial='{data_inicio}'&@dataFinalCotacao='{data_fim}'&$select=dataHoraCotacao,cotacaoCompra&$format=json"
    target_table_name_api = "dm_cotacao_dolar"
    full_target_table_name_api = f"{catalogo_destino}.{schema_destino}.{target_table_name_api}"

    print(f"Buscando cotações de {data_inicio} até {data_fim} ... ", end="")
    
    try:
        #EXTRAIR
        response = requests.get(api_url)
        response.raise_for_status() #Gera erro se a requisição falhar
        data = response.json()
        
        #TRANSFORMAR
        df_pandas = pd.DataFrame(data['value'])
        
        if df_pandas.empty:
            print(f"FALHA. Nenhum dado retornado pela API para o período.")
            return 

        df_spark = spark.createDataFrame(df_pandas)
        df_spark_with_timestamp = df_spark.withColumn("ingestion_timestamp", F.current_timestamp())
        
        #CARREGAR
        (
            df_spark_with_timestamp.write
            .mode("overwrite")
            .format("delta")
            .saveAsTable(full_target_table_name_api)
        )
        
        print(f"SUCESSO.\nTabela {full_target_table_name_api} criada/atualizada.")
        
    except Exception as e:
        print(f"FALHA ao buscar ou processar dados da API. Erro: {str(e)}")

In [0]:
dbutils.widgets.text("data_inicio", "01-01-2017", "Data Inicial (MM-DD-AAAA)")
dbutils.widgets.text("data_fim", "12-31-2018", "Data Final (MM-DD-AAAA)")

#Pegando as datas dos widgets
data_inicio_widget = dbutils.widgets.get("data_inicio")
data_fim_widget = dbutils.widgets.get("data_fim")

#Chamando a função de ingestão da API
ingest_cotacao_dolar_to_bronze(
    data_inicio=data_inicio_widget, 
    data_fim=data_fim_widget, 
    catalogo_destino=catalogo,      
    schema_destino=schema_bronze    
)

In [0]:
#Verificação Final

print("Verificando a tabela da API do Dólar...")

tabela_api_nome = f"{catalogo}.{schema_bronze}.dm_cotacao_dolar"

try:
    #Lê a tabela Delta
    df_api = spark.read.table(tabela_api_nome)
    
    #Exibe os 5 primeiros registros
    display(df_api.limit(5))
    
    print(f"SUCESSO: Tabela {tabela_api_nome} encontrada e lida.")
    
except Exception as e:
    print(f"FALHA ao tentar ler a tabela {tabela_api_nome}. Erro: {str(e)}")